#  Введение в рекуррентные нейронные сети

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Николенко С., Кадурин А., Архангельская Е. Глубокое обучение.
* https://pytorch.org/docs/stable/nn.html#recurrent-layers
* https://karpathy.github.io/2015/05/21/rnn-effectiveness/
* https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html
* https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/
* https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html

## Задачи для совместного разбора

1\. Рассмотрите пример работы одного шага простейшего рекуррентного слоя.  

$$ h' = tanh(W_{ih}x + W_{hh}h) $$

![RNN](https://kvitajakub.github.io/img/rnn-unrolled.png)

In [ ]:
import torch as th
import torch.nn as nn
import torch.optim as optim

In [ ]:
batch_size = 16
seq_len = 8
emb_dim = 32
hidden_dim = 10

x = th.rand(batch_size, seq_len, emb_dim)

cell = nn.RNNCell(input_size=emb_dim, hidden_size=hidden_dim) # A

In [ ]:
# h = zeros
for s in range(seq_len):
  x_i = x[:, s, :]
  h = cell(x_i, h)
  break

In [ ]:
h.shape

torch.Size([16, 10])

2\. Рассмотрите пример работы рекуррентных слоев из `torch.nn`.

In [ ]:
batch_size = 16
seq_len = 8
emb_dim = 32
hidden_dim = 10

x = th.rand(batch_size, seq_len, emb_dim)

In [ ]:
layer = nn.RNN(
    input_size=emb_dim,
    hidden_size=hidden_dim,
    batch_first=True
)

In [ ]:
out, h = layer(x)
out.shape, h.shape

(torch.Size([16, 8, 10]), torch.Size([1, 16, 10]))

In [ ]:
out[0, -1, :]

tensor([ 0.3135,  0.4715,  0.4014,  0.1249,  0.9174, -0.1917, -0.4546,  0.3547,
        -0.0828,  0.6071], grad_fn=<SliceBackward0>)

In [ ]:
h[0, 0]

tensor([ 0.3135,  0.4715,  0.4014,  0.1249,  0.9174, -0.1917, -0.4546,  0.3547,
        -0.0828,  0.6071], grad_fn=<SelectBackward0>)

## Задачи для самостоятельного решения

In [1]:
import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataloader import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.transforms as T
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

<p class="task" id="1"></p>

1\. Используя класс `nn.RNNCell` (абстракцию для отдельного временного шага RNN), реализуйте простейшую рекуррентную сеть Элмана в виде класса `RNN`. Предусмотрите возможность работы с двумя вариантами данных: где данные (x) представлены в виде (batch, seq, feature) и где данные представлены в формате (seq, batch, feature). Создайте тензор `x1` размера 16 x 8 x 32 (batch, seq, feature) и пропустите через модель `RNN`. Выведите на экран форму двух полученных тензоров. Проверьте, что тензор `output[-1]` поэлементно равен `h`.

- [ ] Проверено на семинаре

In [2]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, batch_first):
    super().__init__()
    self.cell = nn.RNNCell(
        input_size=input_size,
        hidden_size=hidden_size
    )
    self.b_f = batch_first

  def forward(self, x, h=None):
    '''
    x.shape = (batch_size, seq_len, feature_size) - тензор входных данных
    h.shape = (batch_size, hidden_size) - тензор со скрытым состоянием RNN
    '''
    outputs = []
    if self.b_f:
      seq_len = x.shape[1]
    else:
      x = x.permute(1, 0, 2)
      seq_len = x.shape[1]
    # инициализация тензора скрытых состояний
    if h is None:
      self.h = th.zeros(size=(x.shape[0], self.cell.hidden_size))

    # проход по каждому элементу последовательностей s в батче и обновление скрытого состояния
    for s in range(seq_len):
      x_i = x[:, s, :]
      self.h = self.cell(x_i, self.h)
      outputs.append(self.h)

    outputs = th.stack(outputs, dim=1)
    # вернуть тензор всех наблюдавшихся скрытых состояний размера (batch_size, seq_len, hidden_size) и тензор скрытых состояний в последний момент времени
    return outputs, self.h

In [3]:
batch_size = 16
seq_len = 8
feature_size = 32
x1 = th.rand(batch_size, seq_len, feature_size)
rnn_model = RNN(input_size=feature_size, hidden_size=10, batch_first=True)
o, h = rnn_model(x1)

In [4]:
o.shape, h.shape

(torch.Size([16, 8, 10]), torch.Size([16, 10]))

In [5]:
th.equal(o[:, -1, :], h)

True

<p class="task" id="2"></p>

2\. Создайте тензор `x2` размера 8 x 16 x 32 (seq, batch, feature) и пропустите через модель `RNN`. Выведите на экран форму двух полученных тензоров. Проверьте, что тензор `output[-1]` поэлементно равен `h`.

- [ ] Проверено на семинаре

In [6]:
x2 = th.rand(seq_len, batch_size, feature_size)
rnn_model = RNN(input_size=feature_size, hidden_size=10, batch_first=False)
o, h = rnn_model(x2)

In [7]:
o.shape, h.shape

(torch.Size([16, 8, 10]), torch.Size([16, 10]))

In [8]:
th.equal(o[:, -1, :], h)

True

<p class="task" id="3"></p>

3\. Считайте файл `pets.csv`, приведите имена питомцев к нижнем регистру. Решите проблему с противоречивостью данных (некоторые имена встречаются в обоих классах). Разбейте набор данных на обучающую и тестовую выборку.  Создайте Vocab на основе обучающей выборки (токен - __буква__). Добавьте в словарь специальный токен `<PAD>`. Выведите на экран количество токенов в полученном словаре.

- [ ] Проверено на семинаре

In [9]:
df = pd.read_csv('pets.csv')
df.columns = ['name', 'class']
df.head()

,name,class
0,Арчи,собака
1,Алекс,собака
2,Амур,собака
3,Алтaй,собака
4,Альф,собака


In [10]:
cnt = df['name'].value_counts()

In [11]:
df_new = df.copy()
for name in cnt[cnt > 1].index:
  cls_del = df[df['name'] == name]['class'].value_counts().index[-1]
  df_new.drop(index=df[(df['name'] == name) & (df['class'] == cls_del)].index, inplace=True)

df.shape, df_new.shape

((3228, 2), (2800, 2))

In [12]:
df_new['name'] = df_new['name'].apply(lambda x: x.lower())

In [13]:
train, test = train_test_split(df_new, test_size=0.2, random_state=42)
train.shape, test.shape

((2240, 2), (560, 2))

In [14]:
vocab = build_vocab_from_iterator(train['name'].values, specials=['<PAD>', '<UNK>'])
vocab.set_default_index(vocab['<UNK>'])

In [15]:
len(vocab)

39

<p class="task" id="4"></p>

4\. Создайте класс `PetsDataset`. Используя преобразования, сделайте длины наборов индексов одинаковой фиксированной длины (подходящее значение определите сами). Закодируйте целыми числами классы питомцев. Создайте два объекта класса `PetsDataset` (для обучающей и тестовой выборки). Выведите на экран их длины.

- [ ] Проверено на семинаре

In [16]:
class PetsDataset:
  def __init__(self, data, vcb):
    self.names = data['name'].values
    self.class_ = data['class'].values
    self.vcb = vcb
    self.transforms = T.Sequential(
        T.ToTensor(0),
        T.PadTransform(max_length=22, pad_value=0)
    )

  def __getitem__(self, idx):
    if isinstance(idx, slice):
      res = []
      for word in self.names[idx]:
        res.append(self.vcb.lookup_indices(list(word)))
      return self.transforms(res), th.LongTensor(self.class_[idx])
    else:
      return self.transforms(self.vcb.lookup_indices(list(self.names[idx]))), th.LongTensor([self.class_[idx]])

  def __len__(self):
    return len(self.names)

In [17]:
le = LabelEncoder()
le.fit(train['class'])
train['class'] = le.transform(train['class'])
test['class'] = le.transform(test['class'])

In [18]:
train_dataset = PetsDataset(train, vocab)
test_dataset = PetsDataset(test, vocab)

In [19]:
len(train_dataset), len(test_dataset)

(2240, 560)

In [20]:
train_loader = DataLoader(train_dataset, batch_size=128)
test_loader = DataLoader(test_dataset, batch_size=128)

<p class="task" id="5"></p>

5\. Используя созданный класс `RNN`, решите задачу классификации категорий питомцев по их именам. Выведите на экран отчет по классификации на обучающем и тестовом множестве.

- [ ] Проверено на семинаре

In [21]:
def report(model, loader, adv=False):
  preds = []
  true = []
  for X_b, y_b in loader:
    if adv:
      out = model(X_b)[0, :, :]
    else:
      out = model(X_b)
    preds.extend(th.argmax(out, dim=1).detach().numpy())
    true.extend(y_b.numpy())
  print(classification_report(true, preds))

In [22]:
class Classifier(nn.Module):
  def __init__(self, vcb):
    super().__init__()
    self.rnn = RNN(input_size=200, hidden_size=100, batch_first=True)
    self.emb = nn.Embedding(num_embeddings=len(vcb), embedding_dim=200)
    self.relu = nn.LeakyReLU()
    self.fc1 = nn.Linear(in_features=100, out_features=64)
    self.fc2 = nn.Linear(in_features=64, out_features=2)

  def forward(self, X):
    e = self.emb(X)
    out, h = self.rnn(e)
    out = self.fc1(h)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [24]:
%%time
n_epoch = 25
lr = 0.001
model = Classifier(vocab)
optimizer = optim.Adam(model.parameters(), lr=lr)
crit = nn.CrossEntropyLoss()

for epoch in range(1, n_epoch+1):
  for X_b, y_b in train_loader:
    out = model(X_b)
    loss = crit(out, th.flatten(y_b))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  if epoch % 5 == 0:
    print(f'{epoch=} loss={loss.item()}')

print('train:')
report(model, train_loader)
print('test:')
report(model, test_loader)

epoch=5 loss=0.6949655413627625
epoch=10 loss=0.6937655806541443
epoch=15 loss=0.6936002373695374
epoch=20 loss=0.6934977769851685
epoch=25 loss=0.693464994430542
train:
              precision    recall  f1-score   support

           0       1.00      0.00      0.00      1111
           1       0.50      1.00      0.67      1129

    accuracy                           0.50      2240
   macro avg       0.75      0.50      0.34      2240
weighted avg       0.75      0.50      0.34      2240

test:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       260
           1       0.54      1.00      0.70       300

    accuracy                           0.54       560
   macro avg       0.27      0.50      0.35       560
weighted avg       0.29      0.54      0.37       560

CPU times: user 21.3 s, sys: 2.32 s, total: 23.6 s
Wall time: 24.3 s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<p class="task" id="6"></p>

6\. Решите предыщую задачу, заменив собственный модуль `RNN` на модули `nn.RNN`, `nn.LSTM` и `nn.GRU`. Сравните результаты работы.

- [ ] Проверено на семинаре

In [25]:
class Models:
  class Classifier_RNN(nn.Module):
    def __init__(self, vcb):
      super().__init__()
      self.rnn = nn.RNN(input_size=200, hidden_size=100, batch_first=True)
      self.emb = nn.Embedding(num_embeddings=len(vcb), embedding_dim=200)
      self.relu = nn.ReLU()
      self.fc1 = nn.Linear(in_features=100, out_features=64)
      self.fc2 = nn.Linear(in_features=64, out_features=2)

    def forward(self, X):
      e = self.emb(X)
      out, h = self.rnn(e)
      out = self.fc1(h)
      out = self.relu(out)
      out = self.fc2(out)
      return out

  class Classifier_LSTM(nn.Module):
    def __init__(self, vcb):
      super().__init__()
      self.rnn = nn.LSTM(input_size=200, hidden_size=100, batch_first=True)
      self.emb = nn.Embedding(num_embeddings=len(vcb), embedding_dim=200)
      self.relu = nn.ReLU()
      self.fc1 = nn.Linear(in_features=100, out_features=64)
      self.fc2 = nn.Linear(in_features=64, out_features=2)

    def forward(self, X):
      e = self.emb(X)
      out, (h, c) = self.rnn(e)
      out = self.fc1(h)
      out = self.relu(out)
      out = self.fc2(out)
      return out

  class Classifier_GRU(nn.Module):
    def __init__(self, vcb):
      super().__init__()
      self.rnn = nn.GRU(input_size=200, hidden_size=100, batch_first=True)
      self.emb = nn.Embedding(num_embeddings=len(vcb), embedding_dim=200)
      self.relu = nn.ReLU()
      self.fc1 = nn.Linear(in_features=100, out_features=64)
      self.fc2 = nn.Linear(in_features=64, out_features=2)

    def forward(self, X):
      e = self.emb(X)
      out, h = self.rnn(e)
      out = self.fc1(h)
      out = self.relu(out)
      out = self.fc2(out)
      return out

In [26]:
def train_model(model, dataset, lr=0.01, n_epoch=25, vcb=vocab):
  optimizer = optim.Adam(model.parameters(), lr=lr)
  crit = nn.CrossEntropyLoss()

  for epoch in range(1, n_epoch+1):
    for X_b, y_b in dataset:
      out = model(X_b)[0, :, :]
      loss = crit(out, th.flatten(y_b))
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    if epoch == 25:
      print(f'{epoch=} loss={loss.item()}')

  return model

In [27]:
models = [Models.Classifier_RNN(vocab), Models.Classifier_LSTM(vocab), Models.Classifier_GRU(vocab)]

In [28]:
%%time
for m in models:
  m = train_model(m, train_loader)
  print(f'train {m}:')
  report(m, train_loader, adv=True)
  print(f'test {m}:')
  report(m, test_loader, adv=True)

epoch=25 loss=0.6932187676429749
train Classifier_RNN(
  (rnn): RNN(200, 100, batch_first=True)
  (emb): Embedding(39, 200)
  (relu): ReLU()
  (fc1): Linear(in_features=100, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
):
              precision    recall  f1-score   support

           0       1.00      0.00      0.00      1111
           1       0.50      1.00      0.67      1129

    accuracy                           0.50      2240
   macro avg       0.75      0.50      0.34      2240
weighted avg       0.75      0.50      0.34      2240

test Classifier_RNN(
  (rnn): RNN(200, 100, batch_first=True)
  (emb): Embedding(39, 200)
  (relu): ReLU()
  (fc1): Linear(in_features=100, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       260
           1       0.54      1.00      0.70       300

    accuracy   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epoch=25 loss=0.5321725010871887
train Classifier_LSTM(
  (rnn): LSTM(200, 100, batch_first=True)
  (emb): Embedding(39, 200)
  (relu): ReLU()
  (fc1): Linear(in_features=100, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
):
              precision    recall  f1-score   support

           0       0.72      0.87      0.79      1111
           1       0.84      0.66      0.74      1129

    accuracy                           0.77      2240
   macro avg       0.78      0.77      0.76      2240
weighted avg       0.78      0.77      0.76      2240

test Classifier_LSTM(
  (rnn): LSTM(200, 100, batch_first=True)
  (emb): Embedding(39, 200)
  (relu): ReLU()
  (fc1): Linear(in_features=100, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
):
              precision    recall  f1-score   support

           0       0.65      0.81      0.72       260
           1       0.79      0.62      0.69       300

    accurac

## Обратная связь
- [ ] Хочу получить обратную связь по решению